# Byte-Pair Encoding tokenization

Install the Transformers, Datasets, and Evaluate libraries to run this notebook.

In [1]:
!pip install datasets evaluate transformers[sentencepiece]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 485.4/485.4 kB 9.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 10.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 10.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 11.6 MB/s eta 0:00:00


In [2]:
corpus = [
    "ChatGPT is a sibling model to InstructGPT, which is trained to follow an instruction in a prompt and provide a detailed response.",
    "The metaverse means different things to different people.",
]

In [3]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("gpt2")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

In [4]:
from collections import defaultdict

word_freqs = defaultdict(int)

for text in corpus:
    words_with_offsets = tokenizer.backend_tokenizer.pre_tokenizer.pre_tokenize_str(text)
    new_words = [word for word, offset in words_with_offsets]
    for word in new_words:
        word_freqs[word] += 1

print(word_freqs)

defaultdict(<class 'int'>, {'ChatGPT': 1, 'Ġis': 2, 'Ġa': 3, 'Ġsibling': 1, 'Ġmodel': 1, 'Ġto': 3, 'ĠInstructGPT': 1, ',': 1, 'Ġwhich': 1, 'Ġtrained': 1, 'Ġfollow': 1, 'Ġan': 1, 'Ġinstruction': 1, 'Ġin': 1, 'Ġprompt': 1, 'Ġand': 1, 'Ġprovide': 1, 'Ġdetailed': 1, 'Ġresponse': 1, '.': 2, 'The': 1, 'Ġmetaverse': 1, 'Ġmeans': 1, 'Ġdifferent': 2, 'Ġthings': 1, 'Ġpeople': 1})


In [5]:
alphabet = []

for word in word_freqs.keys():
    for letter in word:
        if letter not in alphabet:
            alphabet.append(letter)
alphabet.sort()

print(alphabet)

[',', '.', 'C', 'G', 'I', 'P', 'T', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'l', 'm', 'n', 'o', 'p', 'r', 's', 't', 'u', 'v', 'w', 'Ġ']


In [6]:
vocab = ["<|endoftext|>"] + alphabet.copy()

In [7]:
splits = {word: [c for c in word] for word in word_freqs.keys()}

In [8]:
def compute_pair_freqs(splits):
    pair_freqs = defaultdict(int)
    for word, freq in word_freqs.items():
        split = splits[word]
        if len(split) == 1:
            continue
        for i in range(len(split) - 1):
            pair = (split[i], split[i + 1])
            pair_freqs[pair] += freq
    return pair_freqs

In [9]:
pair_freqs = compute_pair_freqs(splits)

for i, key in enumerate(pair_freqs.keys()):
    print(f"{key}: {pair_freqs[key]}")
    if i >= 5:
        break

('C', 'h'): 1
('h', 'a'): 1
('a', 't'): 1
('t', 'G'): 2
('G', 'P'): 2
('P', 'T'): 2


In [10]:
best_pair = ""
max_freq = None

for pair, freq in pair_freqs.items():
    if max_freq is None or max_freq < freq:
        best_pair = pair
        max_freq = freq

print(best_pair, max_freq)

('Ġ', 'a') 5


In [11]:
merges = {("Ġ", "t"): "Ġt"}
vocab.append("Ġt")

In [12]:
def merge_pair(a, b, splits):
    for word in word_freqs:
        split = splits[word]
        if len(split) == 1:
            continue

        i = 0
        while i < len(split) - 1:
            if split[i] == a and split[i + 1] == b:
                split = split[:i] + [a + b] + split[i + 2 :]
            else:
                i += 1
        splits[word] = split
    return splits

In [13]:
splits = merge_pair("Ġ", "t", splits)
print(splits["Ġtrained"])

['Ġt', 'r', 'a', 'i', 'n', 'e', 'd']


In [14]:
vocab_size = 50

while len(vocab) < vocab_size:
    pair_freqs = compute_pair_freqs(splits)
    best_pair = ""
    max_freq = None
    for pair, freq in pair_freqs.items():
        if max_freq is None or max_freq < freq:
            best_pair = pair
            max_freq = freq
    splits = merge_pair(*best_pair, splits)
    merges[best_pair] = best_pair[0] + best_pair[1]
    vocab.append(best_pair[0] + best_pair[1])

In [15]:
print(merges)

{('Ġ', 't'): 'Ġt', ('Ġ', 'a'): 'Ġa', ('i', 'n'): 'in', ('Ġ', 'm'): 'Ġm', ('d', 'e'): 'de', ('Ġt', 'o'): 'Ġto', ('n', 's'): 'ns', ('Ġ', 'p'): 'Ġp', ('r', 'e'): 're', ('t', 'G'): 'tG', ('tG', 'P'): 'tGP', ('tGP', 'T'): 'tGPT', ('Ġ', 'i'): 'Ġi', ('Ġi', 's'): 'Ġis', ('in', 'g'): 'ing', ('t', 'r'): 'tr', ('tr', 'u'): 'tru', ('tru', 'c'): 'truc', ('e', 'd'): 'ed', ('Ġa', 'n'): 'Ġan', ('Ġ', 'in'): 'Ġin'}


In [16]:
print(vocab)

['<|endoftext|>', ',', '.', 'C', 'G', 'I', 'P', 'T', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'l', 'm', 'n', 'o', 'p', 'r', 's', 't', 'u', 'v', 'w', 'Ġ', 'Ġt', 'Ġa', 'in', 'Ġm', 'de', 'Ġto', 'ns', 'Ġp', 're', 'tG', 'tGP', 'tGPT', 'Ġi', 'Ġis', 'ing', 'tr', 'tru', 'truc', 'ed', 'Ġan', 'Ġin']


In [17]:
def tokenize(text):
    pre_tokenize_result = tokenizer._tokenizer.pre_tokenizer.pre_tokenize_str(text)
    pre_tokenized_text = [word for word, offset in pre_tokenize_result]
    splits = [[l for l in word] for word in pre_tokenized_text]
    for pair, merge in merges.items():
        for idx, split in enumerate(splits):
            i = 0
            while i < len(split) - 1:
                if split[i] == pair[0] and split[i + 1] == pair[1]:
                    split = split[:i] + [merge] + split[i + 2 :]
                else:
                    i += 1
            splits[idx] = split

    return sum(splits, [])

In [18]:
tokenize("This is not a token.")

['T',
 'h',
 'i',
 's',
 'Ġis',
 'Ġ',
 'n',
 'o',
 't',
 'Ġa',
 'Ġto',
 'k',
 'e',
 'n',
 '.']